# Verificação de GPU e Configuração de Dispositivo
Neste trecho inicial, estamos verificando se temos uma GPU disponível para acelerar o processamento.
Caso uma GPU esteja disponível, o código utiliza o dispositivo `cuda`; caso contrário, o processamento será feito na CPU.

In [ ]:
# vendo se temos GPU disponível

# !nvidia-smi <-- não vai funcionar se não tiver GPU!

import time
import numpy as np
import torch
import transformers

print(torch.cuda.is_available()) # retorna True se encontrou uma GPU para mandarmos jobs

# dinamicamente identificando o dispositivo
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

# Carregando o modelo de linguagem TinyLlama
Aqui utilizamos o pipeline da biblioteca `transformers` para carregar o modelo `TinyLlama-1.1B-Chat-v1.0`. O modelo é carregado no dispositivo previamente identificado (GPU ou CPU).

In [ ]:
llama = transformers.pipeline(
    task='text-generation',
    model='TinyLlama/TinyLlama-1.1B-Chat-v1.0',
    device=device
)

print(llama)
print(llama.model)

# Geração de Texto com o Modelo
Neste passo, o modelo é utilizado para gerar texto com base no `prompt` dado. Definimos o número máximo de novos tokens para a resposta.

In [ ]:
prompt = "How are you?"

output = llama(prompt, max_new_tokens=25)


print(output)
print(output[0]['generated_text']) 

# Codificação do Prompt
O prompt de entrada é convertido em IDs de tokens. Esses IDs são então enviados para o dispositivo e utilizados pelo modelo para gerar a sequência de saída.

In [ ]:
prompt_input_ids = llama.tokenizer.encode(prompt, return_tensors='pt').to(device)
print(prompt_input_ids)

output = llama.model.generate(
    prompt_input_ids, max_new_tokens=25,
    return_dict_in_generate=True,
    output_scores=True, 
)

print(output.keys())

# Computando as Pontuações de Transição
Após a geração de tokens, o código calcula as pontuações de transição para cada token gerado, utilizando os logits gerados pelo modelo.

In [ ]:

transitions = llama.model.compute_transition_scores(
    output.sequences, output.scores, normalize_logits=True
)

transitions

# Análise dos Tokens Gerados
Aqui, o código exibe as pontuações dos tokens gerados após o prompt, ignorando o prompt original. Além disso, ele mostra a probabilidade associada a cada token gerado.

In [ ]:
tamanho_prompt = len(prompt_input_ids[0])
print(tamanho_prompt)

# nosso prompt faz parte da resposta gerada!
print(output.sequences[0][:tamanho_prompt])

# Pegando soh o que foi gerado
generated_tokens = output.sequences[0][tamanho_prompt:]
print(generated_tokens)

print(' token id | score  | token str | prob % ')
for (token, score) in zip(generated_tokens, transitions[0]):
    if llama.tokenizer.decode(token) == '\n':
      continue

    print(f"| {token:8d} | {score.to('cpu').numpy():.3f} | {llama.tokenizer.decode(token):9s} | {np.exp(score.to('cpu').numpy()):.4f} |")

# Medindo o Tempo de Execução
Nesta célula, é medido o tempo que o modelo leva para gerar a saída com 100 novos tokens.

In [ ]:
start = time.time()

output = llama(prompt, max_new_tokens=100)

end = time.time()

print(llama.model)
print(f'Tempo executado: {end - start:.2f} ')
print(output)
print(output[0]['generated_text'])

# Gerando Respostas para Vários Prompts
Esta célula faz uma interação com dois prompts diferentes e compara as respostas do modelo. Aqui, estamos testando a memória da conversa.

In [ ]:
# Lembrando da conversa

prompt1 = "What day is today?"
prompt2 = "What day is tomorrow?"

output = llama(prompt1, max_new_tokens=10)
print(output[0]['generated_text'])

print("-"*80)

output = llama(prompt2, max_new_tokens=10)
print(output[0]['generated_text'])


# Testando a Continuidade da Conversa
O prompt de saída do primeiro comando é utilizado como entrada para o segundo comando, junto com um novo prompt, para ver como o modelo continua a conversa.

In [ ]:
prompt1 = "What day is today?"
prompt2 = "What day is tomorrow?"

output = llama(prompt1, max_new_tokens=10)
print(output[0]['generated_text'])

print("-"*80)

output = llama(output[0]['generated_text'] + prompt2, max_new_tokens=30)
print(output[0]['generated_text'])